In [1]:
import scrapy
from scrapy import Request

In [ ]:
class Departamento(scrapy.Item):
# Define aquí los campos que te interesen
    nombre = scrapy.Field()
    precio = scrapy.Field()
    url = scrapy.Field()
    direccion = scrapy.Field()
    m2 = scrapy.Field()
    habitaciones = scrapy.Field()
    banos= scrapy.Field()
    parqueo= scrapy.Field()
    sector= scrapy.Field()
    alicuota= scrapy.Field()

class Plusvalia(scrapy.Spider):
    name = 'Plusvalia'
    allowed_domains = ['plusvalia.com']
    # punto(s) de partida — páginas horizontales (categorías, menús)
    start_urls = [
    'https://www.example.com/categories',
    # añadir más categorías o landing pages si aplica
    ]
    custom_settings = {
    # Ajustes básicos (puedes moverlos a settings.py)
    'CONCURRENT_REQUESTS': 8,
    'DOWNLOAD_DELAY': 0.5,
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    }


In [4]:
from scrapy.spiders import Rule
from scrapy.linkextractors import LinkExtractor

In [ ]:
rules= {
    Rule(
        LinkExtractor(
            allow=r'/'
        ),follow=True,callback="parse_depa"
    )
}

In [7]:
from scrapy.selector import Selector
from scrapy.loader import ItemLoader

In [ ]:
def parse_depa(self,response):
    sel= Selector(response)
    item= ItemLoader(Departamento(),sel)
    #item.add_xpath("nombre","")
    item.add_xpath("precio",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[1]/div/div/div/div[1]/div')
    #item.add_xpath("url","")
    item.add_xpath("m2",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[3]/h3/span[1]')
    item.add_xpath("direccion",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[2]/div/div')
    //*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[2]/div/div
    item.add_xpath("habitaciones",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[3]/h3/span[2]')
    item.add_xpath("banos",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[3]/h3/span[3]')
    item.add_xpath("parqueo",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[3]/h3/span[4]')
    item.add_xpath("sector",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[2]/div/h2')
    item.add_xpath("alicuota",'//*[@id="root"]/div[2]/div/div/div[3]/div/div[2]/div[26]/div/div[1]/div[3]/div[1]/div/div[1]/div/div/div/div[2]')
    yield item.load_item()

In [ ]:



def parse(self, response):
    """
    Parse horizontal: extrae enlaces a categorías/subcategorías y páginas de listado.
    También puede extraer enlaces directos a productos si aparecen aquí.
    """
# 1) Extraer enlaces a subcategorías (horizontalidad)
    category_links = response.css('a.category-link::attr(href)').getall()
    for href in category_links:
        yield response.follow(href, callback=self.parse)


# 2) Extraer páginas de listado dentro de la categoría (también horizontalidad)
    list_links = response.css('a.listing-link::attr(href)').getall()
    for href in list_links:
        yield response.follow(href, callback=self.parse_list)


# 3) Si hay productos directamente en esta página, seguir al detalle (verticalidad)
    product_links = response.css('a.product-link::attr(href)').getall()
    for href in product_links:
        yield response.follow(href, callback=self.parse_detail)


def parse_list(self, response):
    """
    Parse páginas de listado: extrae items en esa página (links a detalle) y maneja paginación.
    """
    # Extraer enlaces a productos (verticalidad)
    product_links = response.css('a.product-link::attr(href)').getall()
    for href in product_links:
        yield response.follow(href, callback=self.parse_detail)


    # --- PAGINACIÓN ---
    # 1) Enlaces tipo 'siguiente'
    next_page = response.css('a.next::attr(href)').get()
    if next_page:
        yield response.follow(next_page, callback=self.parse_list)


# 2) Paginación con parámetros numéricos (?page=2)
# Si quieres controlar hasta X páginas o detectar el último, puedes parsear el número
# Ejemplo simple — extraer todos los enlaces numerados